# L5: Agentic RAG & External Memory


In [ ]:
pip install letta==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of locust to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB

In [ ]:
!pip show letta

Name: letta
Version: 0.1.4
Summary: Create LLM agents with long-term memory and custom tools
Home-page: 
Author: Letta Team
Author-email: hi@letta.ai
License: Apache License
Location: /usr/local/lib/python3.10/dist-packages
Requires: chromadb, demjson3, docstring-parser, docx2txt, html2text, httpx, httpx-sse, jinja2, llama-index, llama-index-embeddings-openai, locust, nltk, numpy, prettytable, pydantic, pydantic-settings, python-box, python-multipart, pytz, pyyaml, questionary, setuptools, sqlalchemy, sqlalchemy-json, sqlalchemy-utils, sqlmodel, tiktoken, tqdm, typer
Required-by: 


In [ ]:
!rm  -f ~/.letta/sqlite.db

In [ ]:
# import os
# from dotenv import load_dotenv, find_dotenv
from IPython.display import display, HTML
import json
import html
import re

# # these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
# def load_env():
#     _ = load_dotenv(find_dotenv())

# def get_openai_api_key():
#     load_env()
#     openai_api_key = os.getenv("OPENAI_API_KEY")
#     return openai_api_key



def nb_print(messages):
    html_output = """
    <style>
        .message-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 800px;
            margin: 20px auto;
            background-color: #1e1e1e;
            border-radius: 8px;
            overflow: hidden;
            color: #d4d4d4;
        }
        .message {
            padding: 10px 15px;
            border-bottom: 1px solid #3a3a3a;
        }
        .message:last-child {
            border-bottom: none;
        }
        .title {
            font-weight: bold;
            margin-bottom: 5px;
            color: #ffffff;
            text-transform: uppercase;
            font-size: 0.9em;
        }
        .content {
            background-color: #2d2d2d;
            border-radius: 4px;
            padding: 5px 10px;
            font-family: 'Consolas', 'Courier New', monospace;
            white-space: pre-wrap;
        }
        .status-line {
            margin-bottom: 5px;
            color: #d4d4d4;
        }
        .function-name { color: #569cd6; }
        .json-key { color: #9cdcfe; }
        .json-string { color: #ce9178; }
        .json-number { color: #b5cea8; }
        .json-boolean { color: #569cd6; }
        .internal-monologue { font-style: italic; }
    </style>
    <div class="message-container">
    """

    for msg in messages:
        content = get_formatted_content(msg)

        # don't print empty function returns
        if msg.message_type == "function_return":
            return_data = json.loads(msg.function_return)
            if "message" in return_data and return_data["message"] == "None":
                continue

        title = msg.message_type.replace('_', ' ').upper()
        html_output += f"""
        <div class="message">
            <div class="title">{title}</div>
            {content}
        </div>
        """

    html_output += "</div>"
    display(HTML(html_output))

def get_formatted_content(msg):
    if msg.message_type == "internal_monologue":
        return f'<div class="content"><span class="internal-monologue">{html.escape(msg.internal_monologue)}</span></div>'
    elif msg.message_type == "function_call":
        args = format_json(msg.function_call.arguments)
        return f'<div class="content"><span class="function-name">{html.escape(msg.function_call.name)}</span>({args})</div>'
    elif msg.message_type == "function_return":

        return_value = format_json(msg.function_return)
        #return f'<div class="status-line">Status: {html.escape(msg.status)}</div><div class="content">{return_value}</div>'
        return f'<div class="content">{return_value}</div>'
    elif msg.message_type == "user_message":
        if is_json(msg.message):
            return f'<div class="content">{format_json(msg.message)}</div>'
        else:
            return f'<div class="content">{html.escape(msg.message)}</div>'
    elif msg.message_type in ["assistant_message", "system_message"]:
        return f'<div class="content">{html.escape(msg.message)}</div>'
    else:
        return f'<div class="content">{html.escape(str(msg))}</div>'

def is_json(string):
    try:
        json.loads(string)
        return True
    except ValueError:
        return False

def format_json(json_str):
    try:
        parsed = json.loads(json_str)
        formatted = json.dumps(parsed, indent=2, ensure_ascii=False)
        formatted = formatted.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        formatted = formatted.replace('\n', '<br>').replace('  ', '&nbsp;&nbsp;')
        formatted = re.sub(r'(".*?"):', r'<span class="json-key">\1</span>:', formatted)
        formatted = re.sub(r': (".*?")', r': <span class="json-string">\1</span>', formatted)
        formatted = re.sub(r': (\d+)', r': <span class="json-number">\1</span>', formatted)
        formatted = re.sub(r': (true|false)', r': <span class="json-boolean">\1</span>', formatted)
        return formatted
    except json.JSONDecodeError:
        return html.escape(json_str)

In [ ]:
import openai, os
OPENAI_API_KEY = "your_api_key_here"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY
from letta import create_client

client = create_client()


Saved Config:  /root/.letta/config


In [ ]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))
#client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002"))

In [ ]:
# Loading tools from Langchain
pip install langchain-community==0.2.17

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [ ]:
import getpass
import os
import getpass
import os


os.environ["TAVILY_API_KEY"] = "your_api_key_here"

In [ ]:
from langchain_community.tools import TavilySearchResults
from letta.schemas.tool import Tool

search = TavilySearchResults()

In [ ]:
search.run("What are the three best practices of RFP submission?")

[{'url': 'https://www.bmc.com/blogs/rfp-process/',
  'content': "Best practices for vendors who submit RFPs include: Identify a team lead to own the RFP submission process; Follow the directions as closely as possible; Use your lessons learned from past RFP submissions—both those that were successful and those that didn't win; Include examples of your previous work, like a resume or CV for your company"},
 {'url': 'https://olive.app/blog/rfp-process/',
  'content': 'RFP Process Best Practice: 3 Phases. The RFP process comprises three fundamental phases: creating the RFP, administering it, and evaluating the responses. Creation. The first phase in the process is to create and write an RFP. Planning, stakeholder interviews, research, gathering and ranking rfp requirements are all part of the process. Issue'},
 {'url': 'https://hinzconsulting.com/rfp-creation/',
  'content': 'Submission Method: Provide instructions for submitting proposals, including the preferred method (e.g., email, onl

In [ ]:
# convert the tool to Letta Tool
search_tool = Tool.from_langchain(TavilySearchResults())




In [ ]:
# persist the tool
client.add_tool(search_tool)

Tool(description=None, source_type='python', module=None, user_id='user-461b21e1-4807-4b5b-9ba5-97f71b842320', id='tool-4cf71fda-ce62-489c-9e8d-e631334fcbc5', name='run_tavilysearchresults', tags=['langchain'], source_code="\ndef run_tavilysearchresults(**kwargs):\n    if 'self' in kwargs:\n        del kwargs['self']\n    from langchain_community.tools import TavilySearchResults\n    tool = TavilySearchResults()\n    return tool._run(**kwargs)\n", json_schema={'name': 'run_tavilysearchresults', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'search query to look up'}, 'request_heartbeat': {'type': 'boolean', 'description': "Request an immediate heartbeat after function execution. Set to 'true' if you want to send a follow-up message or run a follow-up f

In [ ]:
research_agent_persona = f"""
You have access to a web via a {search_tool.name} tool.
Use this tool to respond to users' questions, by summarizing the
{search_tool.name}
and also providing the `url` that the information was from as a
reference.

<Example>
User: 'What is Obama's first name?'
Assistant: 'Obama's first name is Barack.

Sources:
[1] https://www.britannica.com/biography/Barack-Obama
[2] https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
</Example>
Your MUST provide URLs that you used to generate the answer, or you will be terminated.

"""

In [ ]:
from letta.schemas.memory import ChatMemory
agent_state = client.create_agent(
    name="research_agent",
    tools=[search_tool.name],
    memory=ChatMemory(
        human="My name is Laksh",
        persona=research_agent_persona
    )
)

In [ ]:
# Loading data into archival memory
source = client.create_source("RFP_sample")
source

ValueError: Source with name RFP_sample already exists for user user-461b21e1-4807-4b5b-9ba5-97f71b842320

In [ ]:
client.load_file_into_source(
    filename="/content/RFP_sample.pdf",
    source_id=source.id,
)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  4.59file/s]


Job(metadata_={}, id='job-8b9b322b-0d22-4532-8809-09b8959b5704', status=<JobStatus.created: 'created'>, created_at=datetime.datetime(2024, 12, 9, 0, 19, 53, 29052, tzinfo=datetime.timezone.utc), completed_at=None, user_id='user-461b21e1-4807-4b5b-9ba5-97f71b842320')

In [ ]:
client.attach_source_to_agent(
    agent_id=agent_state.id,
    source_id=source.id
)

100%|██████████| 1/1 [00:00<00:00, 21.35it/s]


In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message = "What are the three best practices of RFP submission? ",
    role = "user"
)
nb_print(response.messages)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message = "What are the three best practices of RFP submission and use them to generate the approach for the RFP in archival memory ",
    role = "user"
)
nb_print(response.messages)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id,
    message = "Search archival memory for the RFP on Smart Healthcare IoT Platform Integration Project and generate its ask and our approach to respond based on best practices",
    role = "user"
)
nb_print(response.messages)